### Load an Example Graph

In [ ]:
from ipyradiant import FileManager, PathLoader
import rdflib

lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]
rdf_graph = lw.graph
qres = lw.graph.query(
    """
    PREFIX planet: <https://swapi.co/resource/planet/>
    PREFIX human: <https://swapi.co/resource/human/>
    PREFIX film: <https://swapi.co/resource/film/>
    
    CONSTRUCT {
        ?s ?p ?o .
    }
    WHERE {
        ?s ?p ?o .
        
        VALUES (?s) {
            (human:1)
            (human:4)
#            (human:5)
#            (film:1)
#            (film:4)
#            (planet:1)
#            (planet:2)
        }
    }
    """
)

simple_graph = rdflib.graph.Graph().parse(data=qres.serialize(format="xml"))

### GEv2

In [ ]:
import ipycytoscape as cyto
import ipywidgets as W
import traitlets as T
import rdflib

from typing import List
from IPython.display import display, JSON
from pandas import DataFrame
from ipyradiant.query.api import SPARQLQueryFramer, build_values
from ipyradiant.visualization.improved_cytoscape import CytoscapeViewer
from ipyradiant.visualization.explore.interactive_exploration import add_cyto_class, remove_cyto_class
from ipyradiant.visualization.explore import GraphExploreSelectMultiple
from ipyradiant.rdf2nx import RDF2NX


class MetaObjects(type):
    """Metaclass to construct triples for a specified set of triple values."""

    _sparql = """
        SELECT DISTINCT ?o
        WHERE {{
            ?s ?p ?o .
            FILTER (!isLiteral(?o))

            VALUES ({}) {{
                {}
            }}
        }}
    """
    values = None

    @property
    def sparql(cls):
        return build_values(cls._sparql, cls.values)


class TargetObjects(SPARQLQueryFramer, metaclass=MetaObjects):
    values = None


class CustomNodeIRIs:
    # We already know the IRIs, so we need a dummy class to return them when RDF2NX requests them
    iris = tuple()
    
    @classmethod
    def run_query(cls, rdf_graph):
        return DataFrame(cls.iris, columns=["iri"])
    

class ConstructFocusGraphMeta(type):
    """Metaclass to construct a sub-graph of relevant triples."""

    _sparql = """
        CONSTRUCT {{
            ?iri ?predicate ?value .
        
            ?value a ?type;
                ?pred ?object .
        }}
        WHERE {{
            ?iri ?predicate ?value .
            
            OPTIONAL {{
                ?value a ?type ;
                    ?pred ?object .
                
                FILTER (isLiteral(?object))
            }}

            VALUES ({}) {{
                {}
            }}
        }}
    """
    values = None

    @property
    def sparql(cls):
        return build_values(cls._sparql, cls.values)


class ConstructFocusGraph(SPARQLQueryFramer, metaclass=ConstructFocusGraphMeta):
    values = None
    


class GEv2(W.VBox):
    # TODO when rdf_graph changes, reset the widget    
    rdf_graph = T.Instance(rdflib.graph.Graph)  # the graph to query (e.g. RDF2NX)
    sub_graph = T.Instance(rdflib.graph.Graph)  # a sub-graph containing reduces information

    selector = T.Instance(GraphExploreSelectMultiple)
    selector_values = T.List(T.Instance(rdflib.term.URIRef), allow_none=True, default_value=None)
    
    graph_view = T.Instance(CytoscapeViewer)
    selected_node = T.Instance(cyto.Node, allow_none=True)
    
    _rdf2nx_converter = RDF2NX()  # Use instance
    
    _log = W.Output()
    json_output = W.Output()
    
    def get_node(self, node: dict) -> cyto.Node:
        """This function is used to find a node given the id of a node copy"""

        for cyto_node in self.graph_view.cytoscape_widget.graph.nodes:
            if cyto_node.data["id"] == node["data"]["id"]:
                return cyto_node
        raise ValueError("Node not found in cytoscape.graph.nodes.")
        
    def get_node_by_id(self, id_) -> cyto.Node:
        """This function is used to find a node given the id of a node"""

        for cyto_node in self.graph_view.cytoscape_widget.graph.nodes:
            if cyto_node.data["id"] == id_:
                return cyto_node
        raise ValueError("Node not found in cytoscape.graph.nodes.")
        
    @_log.capture()
    def expand_nodes(self, iris: List[rdflib.term.URIRef]):
        # TODO
        # add to the subgraph, do not replace
        # how are we going to revert the sub-graph? can we just remove the node and let networkx clean up?
        ConstructFocusGraph.values = {"iri": iris}
        additional_sub_graph = rdflib.graph.Graph()
        for s, p, o in ConstructFocusGraph.run_query(self.rdf_graph).values:
            additional_sub_graph.add((s, p, o))
        
        self.sub_graph = self.sub_graph + additional_sub_graph
    
    @_log.capture()
    def node_click_action(self, node: dict):
        # Have to get the object by dict reference
        cyto_node = self.get_node(node)
        
        # note: at this point, self.selected_node is the last-clicked node (we can use to enable double-click action)
        if self.selected_node != cyto_node:
            if self.selected_node:
                self.selected_node.classes = remove_cyto_class(self.selected_node, "clicked")
            cyto_node.classes = add_cyto_class(cyto_node, "clicked")
        else:
            if "expanded" in cyto_node.classes:
                # remove the expansion (remove node and let cytoscape clean up?)
                # TODO
                pass
            else:               
                # expand the node
                iri = cyto_node.data.get("iri")
                self.expand_nodes([iri])
                cyto_node.classes = add_cyto_class(cyto_node, "expanded")
        
        self.selected_node = cyto_node
        
    @T.default("selector")
    def _create_selector(self):
        w = GraphExploreSelectMultiple()
        T.link((self, "rdf_graph"), (w, "graph"))
        T.link((w.subject_select.select_widget, "value"), (self, "selector_values"))
        return w
    
    @T.default("graph_view")
    def _create_graph_view(self):
        w = CytoscapeViewer(
            layout=W.Layout(
                width='60%'
            )
        )
        w.cytoscape_widget.on("node", "click", self.node_click_action)
        return w
    
    @T.default("rdf_graph")
    def _create_rdf_graph(self):
        return rdflib.Graph()
    
    @T.default("sub_graph")
    def _create_sub_graph(self):
        return rdflib.Graph()
    
    @T.default("selected_node")
    def _create_default_selected_node(self):
        return None
    
    @T.validate("children")
    def validate_children(self, proposal):
        """
        Validate method for default children.
        This is necessary because @T.default does not work on children.
        """
        children = proposal.value
        if not children:
            children = (
                W.HBox([self.selector, self.graph_view]), 
                self.json_output
            )
        return children
    
    @T.observe("selected_node")
    def load_json(self, change):
        if change.new == change.old:
            return None

        # must be copy to prevent changing the object
        data = dict(change.new.data)
        data.pop("_label", None)  # TODO just remove private and non-serializable
        with self.json_output:
            self.json_output.clear_output()
            display(JSON(data))
            
    @T.observe("selector_values")
    @_log.capture()
    def update_sub_graph(self, change):
        # TODO how to make this wait for a second to see if the user updates again (e.g. click-click-click)
        if change.old == change.new or not change.new:
            return
        
#         # update class iris; this reduces the nodes to capture in the graph
#         TargetObjects.values={"s": change.new}
#         qres = TargetObjects.run_query(self.rdf_graph)  # get the objects as well
#         CustomNodeIRIs.iris = [*self.selector.subject_select.select_widget.value, *qres["o"].values]
#         # modify converter 
#         self._rdf2nx_converter.node_iris = CustomNodeIRIs  # Do we need?

        # attempt 2: run construct query and pass results to default converter
        # Note: this will resullts in lots of INFO messages that edges are being skipped due to missing nodes (expected)
        self.sub_graph = rdflib.graph.Graph()  # reset the graph
        self.expand_nodes(change.new)  # updates self.sub_graph
        
    @T.observe("sub_graph")
    @_log.capture()
    def update_graph_view(self, change):
        print("updating graph view")
        # run converter
        nx_graph = self._rdf2nx_converter.convert(
            rdf_graph=self.sub_graph, 
            namespaces=dict(self.rdf_graph.namespaces())
        )
        # postprocess nx_graph (TODO add comment about labels)
        for node, data in nx_graph.nodes(data=True):
            nx_graph.nodes[node]["label"] = data.get("rdfs:label", "No Label")
        
        # assign to graph_view
        self.graph_view.graph = nx_graph

* [X] Create wrapper class for the new CytoscapeViewer
* [x] Implement single-click behavior
* [x] Integrate with node selector
* [x] Observe selector_values and run RDF2NX to generate sub_graph
  * [x] Update to generate outgoing connections as well as initial node
  * [x] Update to remove full connections for the graph (only want edges from source to target, not target to all other nodes)
* [x] Patch to get proper initial subject node working [partial fix, see comments]
* [ ] Implement double-click behavior
  * [x] Expansion
  * [ ] Collapse
* [ ] Improve style
  * [ ] expanded vs unexpanded node style (what should I double-click?)
  * [ ] height of graph layout
  * [ ] button for execution?
* [ ] Implement property for serialized sub-graph

> Note to self, this is going to require a new converter class that has a custom workflow to separate the focus node(s) from its(their) connections

TODO:
* merge reflexive properties into something different?
* should we replace the automatic graph generation with a button? (very clunky when clicking multiple things one-at-a-time)

ISSUES:
* Expanding a node causes the "clicked" style to disappear (class is still set, just not rendered)
  * This is an issue with the rebuilding of the cytoscapewidget (clears the classes)

In [ ]:
gev2 = GEv2()
gev2.rdf_graph = lw.graph
gev2

In [ ]:
gev2._log

In [ ]:
type(node)

In [ ]:
for node in gev2.graph_view.cytoscape_widget.graph.nodes:
    print(node.data["iri"], node.classes)

In [ ]:
gev2.graph_view

In [ ]:
# removing also breaks the front-end (doesnt refresh)
gev2.graph_view.cytoscape_widget.graph.remove_node(gev2.graph_view.cytoscape_widget.graph.nodes[1])

In [ ]:
gev2.graph_view